# ReWTS-LLM-RL Training su Google Colab

Questo notebook permette di eseguire il training del sistema ReWTSE-LLM-RL su Google Colab.

## Setup Necessario

1. Carica il tuo progetto su Google Drive nella cartella `MyDrive/Papers`
2. Configura la tua API key di Gemini nei Secrets di Colab (🔑)
3. Assicurati di avere i dati preprocessati nella cartella `data/processed/`
4. Abilita GPU: Runtime > Change runtime type > GPU

## 1. Verifica GPU disponibile

In [ ]:
import torch

# Verifica se CUDA è disponibile
if torch.cuda.is_available():
    print(f"✓ GPU disponibile: {torch.cuda.get_device_name(0)}")
    print(f"  Memoria GPU: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠ GPU non disponibile. Il training sarà più lento.")
    print("  Per abilitare GPU: Runtime > Change runtime type > Hardware accelerator > GPU")

## 2. Monta Google Drive

In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')
print("✓ Google Drive montato con successo")

## 3. ⚙️ CONFIGURAZIONE CENTRALIZZATA

**IMPORTANTE:** Modifica questa cella per configurare il training.

### Parametri da modificare:
- `PROJECT_PATH`: Percorso al progetto su Google Drive
- `tickers`: Lista dei ticker da processare
- `chunk_length`: Lunghezza dei chunk temporali
- `episodes_per_chunk`: Numero di episodi per chunk (riduci per training più veloce)

In [ ]:
import os

# ============================================================================
# CONFIGURAZIONE PATHS
# ============================================================================

# Path al progetto su Google Drive (MODIFICA SE NECESSARIO)
PROJECT_PATH = '/content/drive/MyDrive/Papers'

# Cambia working directory
os.chdir(PROJECT_PATH)
print(f"✓ Working directory: {os.getcwd()}")

# ============================================================================
# CONFIGURAZIONE TRAINING
# ============================================================================

TRAINING_CONFIG = {
    # -----------------------------------------------------
    # Tickers da processare
    # -----------------------------------------------------
    'tickers': ['AAPL'],  # Lista dei ticker (es: ['AAPL', 'MSFT', 'GOOGL'])
    
    # -----------------------------------------------------
    # Configurazione LLM (Google Gemini)
    # -----------------------------------------------------
    'llm': {
        'llm_model': 'gemini-2.0-flash-exp',  # Modello Gemini da usare
        'temperature': 0.0,                    # Temperature per generazione (0.0 = deterministico)
        'seed': 49,                            # Seed per riproducibilità
        'gemini_api_key': None                 # Verrà impostato dalla cella API key
    },
    
    # -----------------------------------------------------
    # Configurazione ReWTSE (Reinforcement Learning)
    # -----------------------------------------------------
    'rewts': {
        'chunk_length': 500,           # Lunghezza di ogni chunk temporale (giorni di trading)
        'lookback_length': 100,        # Finestra di lookback per le features
        'forecast_horizon': 1,         # Orizzonte di previsione (giorni)
        'episodes_per_chunk': 20,      # Episodi di training per chunk (riduci per velocità)
        
        # Parametri DDQN
        'gamma': 0.99,                 # Discount factor (quanto valutare reward futuri)
        'epsilon_start': 1.0,          # Epsilon iniziale per exploration
        'epsilon_min': 0.01,           # Epsilon minimo
        'epsilon_decay': 0.995,        # Decay rate di epsilon
        'learning_rate': 1e-3,         # Learning rate per optimizer
        'batch_size': 64,              # Batch size per training
        'buffer_size': 10000,          # Dimensione del replay buffer
        'target_update_freq': 10,      # Frequenza aggiornamento target network
        'hidden_dims': [128, 64]       # Architettura rete neurale (hidden layers)
    },
    
    # -----------------------------------------------------
    # Configurazione Trading Environment
    # -----------------------------------------------------
    'trading_env': {
        'initial_balance': 10000,      # Capitale iniziale ($)
        'transaction_cost': 0.001,     # Costo di transazione (0.1%)
        'max_position': 1.0            # Posizione massima (frazione del capitale)
    },
    
    # -----------------------------------------------------
    # Frequenza generazione strategie LLM
    # -----------------------------------------------------
    'strategy_frequency': 20  # Genera nuova strategia ogni N giorni di trading
}

# ============================================================================
# STAMPA CONFIGURAZIONE
# ============================================================================

print("\n" + "="*60)
print("CONFIGURAZIONE TRAINING")
print("="*60)
print(f"\nProject Path: {PROJECT_PATH}")
print(f"\nTickers: {TRAINING_CONFIG['tickers']}")
print(f"\nLLM:")
print(f"  - Model: {TRAINING_CONFIG['llm']['llm_model']}")
print(f"  - Temperature: {TRAINING_CONFIG['llm']['temperature']}")
print(f"  - Seed: {TRAINING_CONFIG['llm']['seed']}")
print(f"\nReWTSE:")
print(f"  - Chunk length: {TRAINING_CONFIG['rewts']['chunk_length']} giorni")
print(f"  - Episodes per chunk: {TRAINING_CONFIG['rewts']['episodes_per_chunk']}")
print(f"  - Strategy frequency: ogni {TRAINING_CONFIG['strategy_frequency']} giorni")
print(f"\nTrading:")
print(f"  - Initial balance: ${TRAINING_CONFIG['trading_env']['initial_balance']}")
print(f"  - Transaction cost: {TRAINING_CONFIG['trading_env']['transaction_cost']*100}%")
print("\n" + "="*60)

## 4. Installa dipendenze

In [ ]:
# Installa le dipendenze
print("Installazione dipendenze...")
!pip install -q -r requirements.txt

# Verifica installazione
import google.generativeai as genai
print("✓ Tutte le dipendenze installate correttamente")

## 5. Configura API Key

### Setup Sicuro API Key (Raccomandato):

1. Clicca sull'icona della chiave 🔑 nella barra laterale sinistra
2. Aggiungi un nuovo secret: `GEMINI_API_KEY`
3. Incolla la tua API key di Google Gemini
4. Abilita "Notebook access"

**Ottieni la tua API key gratuita:** https://ai.google.dev/

### Metodo Alternativo (File .env):

Se preferisci usare un file `.env` su Drive, decommenta il codice alternativo nella cella sotto.

In [ ]:
from google.colab import userdata
import os

# ============================================================================
# METODO 1: Colab Secrets (RACCOMANDATO)
# ============================================================================

try:
    # Leggi la API key dai secrets di Colab
    GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
    os.environ['GEMINI_API_KEY'] = GEMINI_API_KEY
    
    # Aggiorna config
    TRAINING_CONFIG['llm']['gemini_api_key'] = GEMINI_API_KEY
    
    # Mostra key mascherata per verifica
    masked_key = f"{GEMINI_API_KEY[:10]}...{GEMINI_API_KEY[-4:]}"
    print(f"✓ API key configurata: {masked_key}")
    
except Exception as e:
    print(f"⚠ Errore Colab Secrets: {e}")
    print("\nConfigura GEMINI_API_KEY nei secrets di Colab (icona 🔑)")
    print("Oppure usa il metodo alternativo .env (vedi sotto)")

# ============================================================================
# METODO 2: File .env (ALTERNATIVO)
# ============================================================================
# Decommenta se preferisci usare un file .env su Google Drive

# env_path = os.path.join(PROJECT_PATH, '.env')
# if os.path.exists(env_path):
#     with open(env_path) as f:
#         for line in f:
#             line = line.strip()
#             if line and not line.startswith('#') and '=' in line:
#                 key, value = line.split('=', 1)
#                 os.environ[key] = value
#     
#     GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
#     TRAINING_CONFIG['llm']['gemini_api_key'] = GEMINI_API_KEY
#     print(f"✓ API key caricata da .env: {GEMINI_API_KEY[:10]}...{GEMINI_API_KEY[-4:]}")
# else:
#     print(f"⚠ File .env non trovato in {env_path}")

# ============================================================================
# VERIFICA API KEY
# ============================================================================

if TRAINING_CONFIG['llm']['gemini_api_key']:
    print("\n✅ Configurazione API completata - Puoi procedere")
else:
    print("\n❌ API key non configurata - Configura prima di procedere")

## 6. Verifica dati disponibili

In [ ]:
import os

# Verifica che i dati siano presenti
data_dir = 'data/processed'

print("="*60)
print("VERIFICA DATI")
print("="*60)

if os.path.exists(data_dir):
    files = [f for f in os.listdir(data_dir) if f.endswith('.csv')]
    print(f"\n✓ Trovati {len(files)} file CSV in {data_dir}:")
    
    total_size = 0
    for f in sorted(files):
        file_path = os.path.join(data_dir, f)
        size_mb = os.path.getsize(file_path) / (1024 * 1024)
        total_size += size_mb
        print(f"  - {f:40s} {size_mb:>8.2f} MB")
    
    print(f"\nTotale: {total_size:.2f} MB")
    
    # Verifica che ci siano i file necessari per ogni ticker
    print("\nVerifica file per ticker:")
    all_ok = True
    for ticker in TRAINING_CONFIG['tickers']:
        market_file = f"{ticker}_full_data.csv"
        news_file = f"{ticker}_news.csv"
        
        has_market = market_file in files
        has_news = news_file in files
        
        status = "✓" if (has_market and has_news) else "✗"
        print(f"  {status} {ticker}: Market={has_market}, News={has_news}")
        
        if not (has_market and has_news):
            all_ok = False
    
    if all_ok:
        print("\n✅ Tutti i dati necessari sono presenti")
    else:
        print("\n⚠ ATTENZIONE: Mancano alcuni file necessari")
        
else:
    print(f"\n❌ Directory {data_dir} non trovata!")
    print("\nAssicurati di:")
    print("  1. Aver preprocessato i dati")
    print("  2. Aver caricato la cartella 'data/processed/' su Google Drive")
    print(f"  3. Il PROJECT_PATH sia corretto: {PROJECT_PATH}")

print("="*60)

## 7. Importa moduli del progetto

In [ ]:
import sys
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle

# Aggiungi il progetto al path
if PROJECT_PATH not in sys.path:
    sys.path.insert(0, PROJECT_PATH)

# Importa i moduli del progetto
from src.llm_agents.strategist_agent import StrategistAgent
from src.llm_agents.analyst_agent import AnalystAgent
from src.rl_agents.trading_env import TradingEnv
from src.hybrid_model.ensemble_controller import ReWTSEnsembleController

print("✓ Moduli importati correttamente")

## 8. Funzioni di training

Le seguenti funzioni sono identiche allo script `train_rewts_llm_rl.py`:

In [ ]:
def load_data(ticker, config):
    """Carica dati preprocessati"""
    market_df = pd.read_csv(f"data/processed/{ticker}_full_data.csv", index_col=0, parse_dates=True)
    news_df = pd.read_csv(f"data/processed/{ticker}_news.csv", index_col=0, parse_dates=True)
    
    return market_df, news_df

def precompute_llm_strategies(ticker, market_df, news_df, config):
    """Pre-computa le strategie LLM per tutto il periodo"""
    
    print(f"\n{'='*60}")
    print(f"Pre-computing LLM Strategies for {ticker}")
    print(f"{'='*60}")
    
    strategist = StrategistAgent(config['llm'])
    analyst = AnalystAgent(config['llm'])
    
    strategies = []
    
    # Genera strategie mensili (ogni 20 trading days)
    strategy_frequency = config.get('strategy_frequency', 20)
    num_strategies = len(market_df) // strategy_frequency
    
    for i in tqdm(range(num_strategies), desc="Generating strategies"):
        start_idx = i * strategy_frequency
        end_idx = min((i + 1) * strategy_frequency, len(market_df))
        
        # Dati per questa strategia
        period_data = market_df.iloc[start_idx:end_idx]
        period_news = news_df[
            (news_df.index >= period_data.index[0]) &
            (news_df.index <= period_data.index[-1])
        ]
        
        # Processa news con Analyst Agent
        news_signals = analyst.process_news(period_news.to_dict('records'))
        
        # Prepara input per Strategist
        market_data = {
            'timestamp': str(period_data.index[-1]),
            'Close': float(period_data['Close'].iloc[-1]),
            'Volume': float(period_data['Volume'].iloc[-1]),
            'Weekly_Returns': period_data['Close'].pct_change().tail(20).tolist(),
            'HV_Close': float(period_data['HV_Close'].iloc[-1]) if 'HV_Close' in period_data else 0.0,
            'IV_Close': float(period_data.get('IV_Close', pd.Series([0])).iloc[-1]) if 'IV_Close' in period_data else 0.0,
            'Beta': 1.0,
            'Classification': 'Growth'
        }
        
        fundamentals = {
            'current_ratio': float(period_data.get('Current_Ratio', pd.Series([1.5])).iloc[-1]) if 'Current_Ratio' in period_data else 1.5,
            'debt_to_equity': float(period_data.get('Debt_to_Equity', pd.Series([0.5])).iloc[-1]) if 'Debt_to_Equity' in period_data else 0.5,
            'pe_ratio': float(period_data.get('PE_Ratio', pd.Series([20])).iloc[-1]) if 'PE_Ratio' in period_data else 20.0,
            'gross_margin': float(period_data.get('Gross_Margin', pd.Series([0.4])).iloc[-1]) if 'Gross_Margin' in period_data else 0.4,
            'operating_margin': float(period_data.get('Operating_Margin', pd.Series([0.2])).iloc[-1]) if 'Operating_Margin' in period_data else 0.2,
            'eps_yoy': 0.1,
            'net_income_yoy': 0.1
        }
        
        analytics = {
            'ma_20': float(period_data['SMA_20'].iloc[-1]),
            'ma_50': float(period_data['SMA_50'].iloc[-1]),
            'ma_200': float(period_data['SMA_200'].iloc[-1]),
            'ma_20_slope': float(period_data['SMA_20_Slope'].iloc[-1]),
            'ma_50_slope': float(period_data['SMA_50_Slope'].iloc[-1]),
            'rsi': float(period_data['RSI'].iloc[-1]),
            'macd': float(period_data['MACD'].iloc[-1]),
            'macd_signal': float(period_data['MACD_Signal'].iloc[-1]),
            'atr': float(period_data['ATR'].iloc[-1])
        }
        
        macro_data = {
            'SPX_Close': float(period_data['SPX_Close'].iloc[-1]) if 'SPX_Close' in period_data else 0.0,
            'SPX_Slope': float(period_data['SPX_Close'].diff().iloc[-1]) if 'SPX_Close' in period_data else 0.0,
            'VIX_Close': float(period_data['VIX_Close'].iloc[-1]) if 'VIX_Close' in period_data else 0.0,
            'VIX_Slope': float(period_data['VIX_Close'].diff().iloc[-1]) if 'VIX_Close' in period_data else 0.0,
            'GDP_QoQ': 0.0,
            'PMI': 50.0,
            'PPI_YoY': 0.0,
            'Treasury_YoY': 0.0
        }
        
        # Genera strategia
        last_strategy = strategies[-1] if strategies else None
        
        strategy = strategist.generate_strategy(
            market_data=market_data,
            fundamentals=fundamentals,
            analytics=analytics,
            macro_data=macro_data,
            news_signals=news_signals,
            last_strategy=last_strategy
        )
        
        strategies.append(strategy)
    
    print(f"✓ Generated {len(strategies)} strategies")
    
    # Salva strategies
    os.makedirs('data/llm_strategies', exist_ok=True)
    with open(f"data/llm_strategies/{ticker}_strategies.pkl", 'wb') as f:
        pickle.dump(strategies, f)
    
    return strategies

def train_rewts_ensemble(ticker, market_df, strategies, config):
    """Addestra ReWTSE ensemble di DDQN agents"""
    
    print(f"\n{'='*60}")
    print(f"Training ReWTSE Ensemble for {ticker}")
    print(f"{'='*60}")
    
    # Inizializza ensemble controller
    ensemble = ReWTSEnsembleController(config['rewts'])
    
    # Determina numero di chunks
    chunk_length = config['rewts']['chunk_length']
    num_chunks = len(market_df) // chunk_length
    
    print(f"Total data points: {len(market_df)}")
    print(f"Chunk length: {chunk_length}")
    print(f"Number of chunks: {num_chunks}")
    
    # Train un DDQN per ogni chunk
    for chunk_id in range(num_chunks):
        start_idx = chunk_id * chunk_length
        end_idx = min((chunk_id + 1) * chunk_length, len(market_df))
        
        # Estrai chunk data
        chunk_df = market_df.iloc[start_idx:end_idx].copy()
        
        # Strategie LLM per questo chunk
        strategy_start_idx = start_idx // config['strategy_frequency']
        strategy_end_idx = end_idx // config['strategy_frequency']
        chunk_strategies = strategies[strategy_start_idx:strategy_end_idx]
        
        # Assicurati che ci siano strategie
        if len(chunk_strategies) == 0:
            print(f"Warning: No strategies for chunk {chunk_id}, skipping")
            continue
        
        # Crea environment per il chunk
        env = TradingEnv(chunk_df, chunk_strategies, config['trading_env'])
        
        # Addestra DDQN agent
        agent = ensemble.train_chunk_model(
            chunk_id=chunk_id,
            env=env,
            num_episodes=config['rewts']['episodes_per_chunk']
        )
        
        ensemble.chunk_models.append(agent)
    
    print(f"\n✓ Ensemble training complete!")
    print(f"  Total chunk models: {len(ensemble.chunk_models)}")
    
    return ensemble

print("✓ Funzioni di training definite")

## 9. 🚀 Esegui Training

**ATTENZIONE:** Questa cella eseguirà il training completo.

### Tempi stimati (con GPU T4):
- 1 ticker, chunk_length=500, episodes=20: ~2-4 ore
- Più ticker o parametri più alti: proporzionalmente di più

### Durante il training:
- Il progresso è mostrato con barre `tqdm`
- I modelli sono salvati automaticamente
- Puoi monitorare l'uso GPU con: `!nvidia-smi`

### In caso di disconnessione:
- I dati su Drive sono preservati
- I modelli salvati restano
- Puoi modificare il codice per riprendere da un checkpoint

In [ ]:
# Loop su tutti i ticker
for ticker in TRAINING_CONFIG['tickers']:
    print(f"\n{'#'*60}")
    print(f"# Processing {ticker}")
    print(f"{'#'*60}")
    
    # Load data
    market_df, news_df = load_data(ticker, TRAINING_CONFIG)
    
    # Pre-compute LLM strategies
    strategies = precompute_llm_strategies(ticker, market_df, news_df, TRAINING_CONFIG)
    
    # Train ReWTSE ensemble
    ensemble = train_rewts_ensemble(ticker, market_df, strategies, TRAINING_CONFIG)
    
    # Salva ensemble (crea directory se non esiste)
    os.makedirs('models', exist_ok=True)
    with open(f"models/{ticker}_rewts_ensemble.pkl", 'wb') as f:
        pickle.dump(ensemble, f)
    
    print(f"\n✓ {ticker} complete!")

print(f"\n{'='*60}")
print("✓ All tickers processed successfully!")
print(f"{'='*60}")

## 10. Verifica modelli salvati

In [ ]:
print("="*60)
print("RIEPILOGO MODELLI SALVATI")
print("="*60)

# Verifica che i modelli siano stati salvati
models_dir = 'models'

if os.path.exists(models_dir):
    model_files = [f for f in os.listdir(models_dir) if f.endswith('.pkl')]
    print(f"\n✓ Trovati {len(model_files)} modelli salvati:")
    total_size = 0
    for f in sorted(model_files):
        file_path = os.path.join(models_dir, f)
        size_mb = os.path.getsize(file_path) / (1024 * 1024)
        total_size += size_mb
        print(f"  - {f:40s} {size_mb:>8.2f} MB")
    print(f"\nTotale modelli: {total_size:.2f} MB")
else:
    print("\n⚠ Nessun modello trovato")

# Verifica strategie salvate
strategies_dir = 'data/llm_strategies'
if os.path.exists(strategies_dir):
    strategy_files = [f for f in os.listdir(strategies_dir) if f.endswith('.pkl')]
    print(f"\n✓ Trovate {len(strategy_files)} strategie salvate:")
    total_size = 0
    for f in sorted(strategy_files):
        file_path = os.path.join(strategies_dir, f)
        size_mb = os.path.getsize(file_path) / (1024 * 1024)
        total_size += size_mb
        print(f"  - {f:40s} {size_mb:>8.2f} MB")
    print(f"\nTotale strategie: {total_size:.2f} MB")

print("\n" + "="*60)
print("✅ Training completato con successo!")
print("I modelli sono salvati su Google Drive e rimarranno disponibili.")
print("="*60)

## 11. Download modelli (opzionale)

Se vuoi scaricare i modelli sul tuo computer locale, esegui questa cella.

**Nota:** I modelli restano comunque salvati su Google Drive.

In [ ]:
import shutil
from google.colab import files

# Crea un archivio con i modelli
print("Creazione archivio...")
shutil.make_archive('trained_models', 'zip', 'models')
print("✓ Modelli compressi in trained_models.zip")

# Download del file
print("\nAvvio download...")
files.download('trained_models.zip')
print("✓ Download avviato (controlla i download del browser)")

## 12. Monitora risorse GPU (opzionale)

Esegui questa cella durante il training per monitorare l'uso della GPU.

In [ ]:
!nvidia-smi

## 📝 Note e Suggerimenti

### Risorse GPU Colab
- **Colab Free:** GPU T4 con ~12-16GB RAM, sessioni max 12h
- **Colab Pro:** GPU più potenti (A100), sessioni più lunghe
- Salva periodicamente i checkpoint per sessioni lunghe

### Ottimizzazioni per Training Veloce
- Riduci `episodes_per_chunk`: da 20 a 10-15
- Aumenta `chunk_length`: da 500 a 750-1000
- Aumenta `strategy_frequency`: da 20 a 30-40
- Riduci `buffer_size`: da 10000 a 5000

### Troubleshooting

#### Sessione disconnessa
- I dati su Drive sono preservati
- Puoi riprendere il training manualmente
- Considera di salvare checkpoint più frequenti

#### Out of Memory (OOM)
- Riduci `batch_size`: da 64 a 32
- Riduci `buffer_size`: da 10000 a 5000
- Riduci `chunk_length`: da 500 a 250

#### API Key Error
- Verifica che GEMINI_API_KEY sia configurata
- Controlla quota API su https://ai.google.dev/
- Verifica connessione internet

#### File non trovati
- Verifica che PROJECT_PATH sia corretto
- Assicurati che i dati siano caricati su Drive
- Ri-esegui la cella di mount di Drive

### Best Practices

1. **Testa con 1 ticker** prima di fare training su molti
2. **Monitora la GPU** durante il training
3. **Salva i log** per analisi successive
4. **Fai backup** dei modelli su Drive
5. **Documenta** le configurazioni che funzionano meglio

### Prossimi Passi

Dopo il training:
1. Valuta i modelli su dati di test
2. Analizza le strategie generate
3. Esegui backtesting
4. Confronta con baseline
5. Ottimizza iperparametri